In [1]:
import pandas as pd
import requests
from urllib.parse import quote
import folium

In [2]:
df=pd.read_csv('서울소방서.csv')

In [3]:
df.tail()

,기관명,주소,전화번호
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119


In [6]:
with open('kakao.txt') as file:
    kakao_key=file.read()

In [7]:
lat_list, lng_list = [], []
for i in df.index:
    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df.기관명[i])

In [9]:
df['위도']=lat_list #위도경도값 가져오기
df['경도']=lng_list

In [10]:
df.head()

,기관명,주소,전화번호,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,37.4946721349965,126.91771856513
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,37.57320485877,126.935995916827
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,37.5448260218652,127.082778518406
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,37.4998334512085,127.142591860451
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,37.5302359667761,126.872342277813


In [11]:
df.isnull().sum() #못 가져온 값 있는지 확인

기관명     0
주소      0
전화번호    0
위도      0
경도      0
dtype: int64

지도 그리기

In [12]:
map = folium.Map(location=[37.55, 126.98], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df.기관명[i]}',
        icon=folium.Icon(icon='glyphicon glyphicon-home')
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px"> 서울 소재 소방서 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map